# Argentina programa 4.0 - Módulo 2: Ciencia de Datos

---
## Ejercícios Semana 6. Regresion Polinomial


## Regresión polinomial con datos reales: datos Properati

Volvamos al conjunto de datos Properati, que hemos usado en la semana anterior.

Despues de filtrar bien los datos, hicimos un modelo lienal para predecir el valor de las propiedades basados en su superfície:

precio [USD]=𝜔0+𝜔1⋅sup. total.

* ¿Qué pasa ahora si usamos un modelo polinomial?
* ¿Podemos chequear si un modelo más complejo describirá mejor esos datos?
* ¿Hasta que orden del polinómio tiene sentido ir?



## Otra forma de regularizar: Lasso

Durante la clase, vimos una forma de regularización, conocida por _ridge_ en la que se le agrega a la función error, la suma de los cuadrados de todos los coeficientes, con um peso $\lambda$/2:
$$
E_\text{ridge}(\boldsymbol{\omega}; \lambda) = \frac{1}{2} \sum_{i=1}^{N} \left\{y(x_i, \boldsymbol{\omega}) - t_i\right\}^2
 + \frac{\lambda}{2} \sum_{i=1}^M \omega_i^2\;\;.
$$
El nuevo término es el término de regularización (o penalización).

Como la suma de los cuadrados de todos los coeficientes corresponde al cuadrado de la _norma_ (el módulo) del vector de coeficientes, se dice que esa regularización emplea la _norma_ $l2$.

Otra regresión regularizada que se utiliza a menudo es la regresión **LASSO (_least absolute shrinkage and selection operator_ / operador de reducción y selección mínima absoluta)**, que selecciona de forma natural las variables más relevantes y produce modelos más parsimoniosos.

En lugar de penalizar la función de error utilizando la suma de los cuadrados de los parámetros del modelo, como en el caso anterior, **LASSO** explota la norma $l1$, que es simplemente la suma de los *valores absolutos* de los parámetros del modelo.

En otras palabras, la norma $l1$ de un vector es, simplemente:

$$
||\boldsymbol{\omega}||_1 = \sum_i |\omega_i|\;\;.
$$

La función de error modificada es, por lo tanto,
$$
E_\text{lasso}(\boldsymbol{\omega}; \lambda) = \frac{1}{2} \sum_{i=1}^{N} \left\{y(x_i, \boldsymbol{\omega}) - t_i\right\}^2
 + \frac{\lambda}{2} \sum_{i=1}^M |\omega_i|\;\;,
$$
donde nuevamente introducimos el hiperparámetro $\lambda$ para controlar el nivel de penalización.

###Adiós soluciones analíticas

La primera consecuencia de esta elección de la penalización es que la función de error ya no puede optimizarse (minimizarse) analíticamente. Es necesario recurrir, entonces, a diferentes algoritmos iterativos.

En `sklearn`, hay dos implementaciones:

* `linear_model.Lasso` usa *descenso por coordenadas* para encontrar el mínimo de la función de error.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e3/Coordinate_descent.svg/500px-Coordinate_descent.svg.png" width=500px></img>

* `linear_model.LassoLars` utiliza LARS (regresión de ángulo mínimo / _least angle regression_), estrechamente relacionado con _forward stepwise regression_ (es decir, todos los coeficientes comienzan en cero, $\boldsymbol{\omega} = 0$, y se incrementan progresivamente). Pueden leer más en la [documentación de scikit-learn](https://scikit-learn.org/stable/modules/linear_model.html#least-angle-regression)


### Implementación

Importen `Lasso` y `LassoLars` y exploren su documentación y argumentos. ¿Cuáles son los parámetros que están asociados al procedimiento de optimización?

In [ ]:
from sklearn.linear_model import Lasso, LassoLars

Creen una función `lasso` tal y como hicimos para la regresión de _ridge_. Elijan una implementación de LASSO. Pueden ver los parámetros de Lasso utilizando `Lasso?`.

In [ ]:
Lasso?

In [ ]:
def lasso(m, ll):
    return Pipeline([('poly_features', PolynomialFeatures(degree=m)),
                     ('regressor', Lasso(alpha=ll/2.0, fit_intercept=False, max_iter=500000))])

Utilicen esto para ajustar los datos con *features* polinomiales de grado nueve. Utilicen el mismo parámetro de regularización que el anterior: 0.001.

In [ ]:
lasso_pipe = lasso(9, 0.01)
lasso_pipe.fit(x_train, t_train)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=9)),
                ('regressor',
                 Lasso(alpha=0.005, fit_intercept=False, max_iter=500000))])

Grafiquen los resultados con la ayuda del código de abajo.

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
ax.plot(x_train, t_train, 'o', ms=10, mfc='None', label='Entrenamiento')
ax.plot(x_, lasso_pipe.predict(x_), 'r-', lw=3, alpha=0.8, label='Curva predicha')
ax.plot(x_, ground_truth(x_), 'k-', lw=3, alpha=0.5, label='Modelo real')
    #
ax.set_title('Grado: {}; $\lambda$: {:.2e}'.format(lasso_pipe['poly_features'].degree,
                                                    lasso_pipe['regressor'].alpha *2), fontsize=16)
    #
ax.set_ylim(0, 3.9)
ax.legend(loc=0, fontsize=15)

A primera vista, parece que ambos regresores producen los mismos resultados. Pero en realidad hay una _gran diferencia_ entre ambos métodos.

**Compare los coeficientes encontrados con cada método**. Puede acceder a ellos

Observación: para obtener los coeficientes de la rgreción ridge, van a tener que replicar aquí la parte de rige, como está en el notebook de la clase.

In [ ]:
ridge_pipe['regressor'].coef_

array([[ 1.84607034,  0.19937739, -0.25641525, -0.2297371 , -0.07640904,
         0.08150002,  0.21069503,  0.30738226,  0.37624621,  0.42342855]])

In [ ]:
lasso_pipe['regressor'].coef_

array([ 2.08420373,  0.        , -0.        , -1.30247473, -0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  2.34130064])

**Pregunta**. ¿Ven alguna diferencia?

### Shrinkage recargado

Volvamos a hacer la gráfica de *shrinkage*

In [ ]:
# crea los valores de lambda
lls = np.logspace(-4, 1, 100)

cc_lasso = []

# Itera sobre los valors de lambda, ajusta y guarda los valores de los coeficientes
for ll in lls:
    #print(ll)
    lasso_pipe = lasso(degrees[-1], ll)
    lasso_pipe.fit(x_train, t_train)
    cc_lasso.append(lasso_pipe['regressor'].coef_)

cc_lasso = np.array(cc_lasso)

Grafiquen las amplitudes de los coeficientes en función del parámetro de regularización para el caso de Lasso.

In [ ]:
# Valores de los coeficintes versus el parámetro de renormalización.
fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(111)

for i in range(len(cc_lasso[0])):
    ax.semilogx(lls, cc_lasso[:, i], label='$\omega_{{{}}}$'.format(i), lw=3, alpha=0.6)
ax.legend(ncol=3, fontsize=16)
ax.set_xlabel('$\lambda$')
ax.set_ylabel('Valor del parámetro')

Como pueden ver, a medida que aumentamos el término de regularización, algunos parámetros se van estrictamente a cero. De este modo, la regresión Lasso también funciona como una especie de herramienta de selección automática de modelos.

# Ejercicio avanzado

Hasta ahora, hemos elegido un polinomio de grado nueve y un valor fijo para el parámetro de regularización. Pero, ¿cómo sabemos que estos valores son los óptimos?

Al igual que hicimos con el regresor polinómico no regularizado, podemos utilizar el conjunto de pruebas para evaluar cuál es el valor óptimo del grado *M* y del parámetro de regularización $\lambda$.

* Elija uno de los dos métodos de regularización que hemos discutido anteriormente.
* Haga un bucle sobre los valores de grado y lambda y evalúe el modelo utilizando el conjunto de pruebas.
* Elegir los mejores valores de $M$ y $\lambda$ en base a estos resultados.
* Repita para el otro regularizador.
* Informe de la mejor métrica de rendimiento (MSE) en el conjunto de prueba.
* Prepare un gráfico que muestre el MSE de la prueba en función de los valores de los hiperparámetros. Dado que hay dos hiperparámetros, quizás quieras probar con `plt.imshow` o `plt.pcolor`.

**Pregunta importante**: ¿piensan que ese valor del error cuadrático medio es representativo de lo que obtendría para datos que aún no se han visto en este proceso? ¿Que podría pasar en ese caso?

***